# **Multi-couches**

### Import des librairies

In [ ]:
from sklearn.preprocessing import normalize
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import random

### Charger les données MNIST

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# réduire la taille du train pour accélérer les experiences
nnn=1000
x_train = x_train[:nnn]
y_train = y_train[:nnn]

print("x_train",x_train.shape)
print("Nombre d'image pour entrainer:",x_train.shape[0])
print("Nombre d'image pour tester:",x_test.shape[0])
print("La taille d'une image:",x_train.shape[1:],"ce qui fait au total:",
      x_train.shape[1]*x_train.shape[2],"pixels")
print("La liste des classes:",np.unique(y_train))

### Définition d'une fonction qui transforme en représentation binaire pour plusieurs classes

In [ ]:
def transform_labels(y_train,y_test):
  """
  Cette fonction transforme les classes non-binaires en une représentation binaire
  Par exemple si on a une liste de 6 fleurs chacune peut avoir une des 3 classes
  Entrée: [
           1,
           3,
           3,
           2,
           1,
           2
          ]

  Sortie: [
           [1,0,0], # class 1
           [0,0,1], # class 3
           [0,0,1], # class 3
           [0,1,0], # class 2
           [1,0,0], # class 1
           [0,1,0]  # class 2
          ]
  """

  print('y_train',y_train.shape)
  print('y_test',y_test.shape)

  # concatener train et test
  y_train_test = np.concatenate((y_train,y_test),axis =0)

  # init un encoder Label
  encoder = LabelEncoder()
  # transformer de [1,3,3,2,1,2] à [0,2,2,1,0,1]
  new_y_train_test = encoder.fit_transform(y_train_test)

  # init un encoder one-hot
  encoder = OneHotEncoder()
  # transformer de [0,2,2,1,0,1] à la représentation binaire
  new_y_train_test = encoder.fit_transform(new_y_train_test.reshape(-1,1))

  # resplit the train and test
  new_y_train = new_y_train_test[0:len(y_train)]
  new_y_test = new_y_train_test[len(y_train):]

  print('new_y_train',new_y_train.shape)
  print('new_y_test',new_y_test.shape)

  return new_y_train, new_y_test

### Visualiser quelques images en spécifiant que c'est du noir et blanc

In [ ]:
plt.subplot(221)
plt.imshow(x_train[0], cmap=plt.get_cmap('gray'))
plt.subplot(222)
plt.imshow(x_train[1], cmap=plt.get_cmap('gray'))
plt.subplot(223)
plt.imshow(x_train[2], cmap=plt.get_cmap('gray'))
plt.subplot(224)
plt.imshow(x_train[12], cmap=plt.get_cmap('gray'))
# show the plot
plt.show()

## Entrainer un MLP : Perceptron Multi-Couches

### Transformer les données de `(28,28)` à `(784,1)` - Aplatir l'image en vecteur

In [ ]:
n_train = x_train.shape[0]
x_train = x_train.reshape(n_train,784)

n_test = x_test.shape[0]
x_test = x_test.reshape(n_test,784)

### Normaliser les données pour que chaque pixel soit entre 0 e 1

In [ ]:
x_train = normalize(x_train)
x_test = normalize(x_test)

### Transformer les classes en représentation binaire (one-hot encoding)

In [ ]:
y_train_binaire,y_test_binaire = transform_labels(y_train,y_test)
y_train_binaire = y_train_binaire.toarray()
y_test_binaire = y_test_binaire.toarray()

### Créer la couche d'entrée qui a la même shape que celle d'une instance dans `x_train`

In [ ]:
input_shape = x_train.shape[1:]

input_layer = keras.layers.Input(input_shape)

### Créer une couche cachée qui contient 512 neurones qui prend en entré le input_layer avec une activation `tanh`

In [ ]:
# on utilise Dense car on veut que tous les neurones (de hidden_layer_1) soient connectés à toutes les sorties (de input_layer)
hidden_layer_1 = keras.layers.Dense(units=512, activation='tanh')(input_layer)

### Créer une couche sortie qui contient un nombre de neurones égal au nombre des classes avec une activation `softmax` et qu'elle soit liée à `hidden_layer_1`


In [ ]:
nb_classes = y_train_binaire.shape[1]
output_layer = keras.layers.Dense(units=nb_classes,activation='softmax')(hidden_layer_1)

### Créer maintenant le modèle (qui est un MLP avec une couche cachée)

In [ ]:
model = keras.models.Model(inputs=input_layer, outputs=output_layer)

### Visualiser les informations du modèle

In [ ]:
model.summary()

### Choisir l'algorithme d'optimisation avec un learning rate de 0.1

In [ ]:
learning_rate = 0.1
optimizer_algo = keras.optimizers.SGD(learning_rate=learning_rate)

### Choisir la fonction de coût que l'on veut optimiser: (Categorical Cross-Entropy)

In [ ]:
cost_function = keras.losses.categorical_crossentropy

### Compiler le modèle en lui indiquant que l'on veut mesurer aussi l'accuracy

In [ ]:
model.compile(loss=cost_function,optimizer=optimizer_algo, metrics=['accuracy'])

### Spécifier le fait qu'on veut sauvegarder le meilleur modèle sur le valdiation set

In [ ]:
 model_checkpoint = keras.callbacks.ModelCheckpoint('best-model.h5', monitor='val_loss', save_best_only=True)

## Entrainement

### Choisir le batch size et le nombre d'époques

In [ ]:
mini_batch_size = 256
nb_epochs = 1000

### Entrainer en lui spécifiant d'utiliser une partie du train pour la validation des hyper-paramèteres

In [ ]:
percentage_of_train_as_validation = 0.3
history = model.fit(x_train,y_train_binaire,batch_size=mini_batch_size,
                    epochs=nb_epochs,verbose=False,
                    validation_split=percentage_of_train_as_validation,
                    callbacks=[model_checkpoint])

### Tracer la variation du taux d'erreur sur le train et sur le validation set en fonction du nombre d'epoque

In [ ]:
history_dict = history.history
loss_train_epochs = history_dict['loss']
loss_val_epochs = history_dict['val_loss']

plt.figure()
plt.plot(loss_train_epochs,color='blue',label='train_loss')
plt.plot(loss_val_epochs,color='red',label='val_loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.savefig('epoch-loss.pdf')
plt.show()
plt.close()

### Choisir le modèle sauvegardé

> Bloc en retrait



In [ ]:
model = keras.models.load_model('best-model.h5')

## **Exercices**

### Évaluer l'accuracy de ce meilleur modèle sur le test

In [ ]:
# Votre code ici

 ### Corrigé :

In [ ]:
loss,acc = model.evaluate(x_test,y_test_binaire,verbose=False)

print("L'accuracy sur l'ensemble du test est:",acc)

### Essayer d'améliorer le modèle en variant :


1.   [L'algorithme d'optimisation](https://keras.io/optimizers/)
2.   [Fonctions d'activation](https://keras.io/activations/)
2.   [nombre de couches Dense](https://keras.io/layers/core/)
3.   [Nombre de neurones dans les couches](https://keras.io/layers/core/)
4.   [batch_size](https://keras.io/models/model/#fit)
5.   [taux d'apprentissage](https://keras.io/optimizers)

Créer une liste des valeurs (hyperparamètres) à essayer pour chaque type de variation.
Choisir aléatoirement une combinaison de ces hyperparamèteres.
Répéter vos essais pour trouver celle qui marche le mieux sur le validation set.
Noter l'effet sur le coût de chaque variation d'hyperparamètres (par exemple un très grand learning rate peut mener à un overfitting - un petit nombre de neurones peut mener à un underfitting).
Évaluer votre modèle final sur le test set.

In [ ]:
# Votre code ici

 ### Corrigé :

In [ ]:
# nombre de jeu d'hyperparamètres à essayer
iteration_s = 10

# une liste de learning rate à essayer
learning_rate_s = [2.0, 1.0, 0.1, 0.01, 0.001]
# une liste du batch size à essayer
mini_batch_size_s = [8,16,32,64,128,256,512]

# initialiser l'erreur minimum à l'infini
min_loss = np.inf

print('learning_rate,batch_size,train_loss,val_loss')

# répéter pour un certain nombre d'iterations
for iteration in range(iteration_s):

  # récupérer aléatoirement des hyperparamèters à essayer
  learning_rate =  random.choice(learning_rate_s)
  mini_batch_size =  random.choice(mini_batch_size_s)

  # créer la couche d'entrée qui a la même shape qu'un image dans x_train
  input_shape = x_train.shape[1:]
  input_layer = keras.layers.Input(input_shape)

  # créer la couche cachée de 128 neurones avec l'activation tanh
  # à noter que vous devez changer le nombre de neurones ainsi que
  # l'activation tanh d'une facon dynamique similaire à batch_size etc.
  # n'oubliez pas que la couche cachée doit être liée à la couche d'entrée
  hidden_layer_1 = keras.layers.Dense(units=128, activation='tanh')(input_layer)

  # créer la couche de sortie avec un nombre de neurones égale au nombre
  # de classe ainsi que l'activation softmax qui ne peut pas changer dans ce cas
  nb_classes = y_train_binaire.shape[1]
  output_layer = keras.layers.Dense(units=nb_classes,activation='softmax')(hidden_layer_1)

  # créer le modèle en spécifiant l'input et output
  model = keras.models.Model(inputs=input_layer, outputs=output_layer)

  # choisir l'algorithme d'optimisation (qui doit aussi lui même
  # être choisi d'une manière dynamique) en lui spécifiant le learning rate
  optimizer_algo = keras.optimizers.SGD(lr=learning_rate)

  # choisir la fonction de coût
  cost_function = keras.losses.categorical_crossentropy

  # compiler en lui spécifiant l'accuracy à observer
  model.compile(loss=cost_function,optimizer=optimizer_algo, metrics=['accuracy'])

  # créer le callback model_checkpoint qui se base sur le validation loss
  model_checkpoint = keras.callbacks.ModelCheckpoint('best-model.h5', monitor='val_loss', save_best_only=True)

  # nombre d'epoque
  nb_epochs = 100

  # Entrainer en lui spécifiant d'utiliser 30% du train comme validation set
  percentage_of_train_as_validation = 0.3
  history = model.fit(x_train,y_train_binaire,batch_size=mini_batch_size,
                    epochs=nb_epochs,verbose=False,
                    validation_split=percentage_of_train_as_validation,
                    callbacks=[model_checkpoint])

  # tracer la variation pour savoir s'il y a du overfitting/underfitting
  history_dict = history.history
  loss_train_epochs = history_dict['loss']
  loss_val_epochs = history_dict['val_loss']
  plt.figure()
  plt.plot(loss_train_epochs,color='blue',label='train_loss')
  plt.plot(loss_val_epochs,color='red',label='val_loss')
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.legend()
  plt.savefig('epoch-loss.pdf')
  plt.show()
  plt.close()

  # val_loss
  val_loss = np.array(loss_val_epochs).min()

  # train_loss
  train_loss = loss_train_epochs[np.array(loss_val_epochs).argmin()]

  print(learning_rate,mini_batch_size,train_loss,val_loss)

  # effacer la mémoir de keras
  keras.backend.clear_session()

  # arreter pour chaque iteration pour pouvoir visualiser
  input("Press Enter to continue...")